# Player Baseline - Average

 Baseline - Average

This baseline model computes the average number of points of the previous N years and use it as a prediction to the N+1 year.

In [2]:
# Imports
import pandas as pd
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from datasets.PlayerDataset import *

In [3]:
dataset = get_player_dataset()
print(len(dataset))
player_data_loader = DataLoader(dataset=dataset, batch_size=1) # Batch size of 1 to load one data element at a time


[250.95715528  26.41576048  47.80955488   7.7646803   13.03414528
  20.79882558  -0.45635784  36.71190373   2.35656807   5.61903726
   1.87277077   0.27287226   1.18123822   9.20675656   3.54654197
   0.28084674  79.81955923]
[144.43579635   4.34517805  28.32239301   9.19750169  13.6573494
  21.62137906  10.12232787  42.12998753   2.74911337   6.3513269
   3.27577955   0.72091061   1.73244639   8.66388769   5.97392126
   0.67312371  70.84096832]
3590


In [4]:
print(dataset.__getitem__(1))
print(len(dataset.__getitem__(0)))
print("Features")
print(dataset.__getitem__(1)[0])
print(dataset.__getitem__(1)[0].shape)
print("Target")
print(dataset.__getitem__(1)[1])
print(dataset.__getitem__(1)[1].shape)


(tensor([[-1.6890, -0.3258,  0.7129, -0.1919,  0.7297,  0.3793, -0.0537,  0.6240,
          0.7433, -0.5698,  0.6494, -0.3785, -0.6818,  0.5532,  0.9129, -0.4172,
          0.6378],
        [-1.6890, -0.0957, -0.0286, -0.6268, -0.0757, -0.3144,  0.1439,  0.1730,
          0.0885, -0.7273, -0.2664, -0.3785, -0.6818, -0.7164,  0.7455,  1.0684,
         -0.1951],
        [-1.7029,  0.1345, -1.0172, -0.8442, -0.6615, -0.7770,  0.7366, -0.7765,
         -0.4207, -0.8847, -0.5717, -0.3785, -0.6818, -0.6010, -0.5937, -0.4172,
         -0.8727],
        [-1.6821,  0.3646, -0.4170, -0.7355, -0.5883, -0.6845,  0.0451, -0.5391,
         -0.3116, -0.8847, -0.2664, -0.3785, -0.1046, -0.7164, -0.2589, -0.4172,
         -0.6609],
        [-1.6890,  0.5947,  0.5010, -0.6268,  0.2904, -0.0832,  2.0209, -0.0169,
          0.7433, -0.7273, -0.2664, -0.3785, -0.1046, -0.0239,  0.7455, -0.4172,
         -0.2515]], dtype=torch.float64), tensor([-1.7029,  0.8249,  0.0067, -0.3006, -0.2222, -0.2682,  0.9342, 

In [5]:

# Mean accross columns
x = dataset.__getitem__(1)[0]
print(x)
print(dataset.unnormalize(x))
print(torch.mean(x, dim=0))

tensor([[-1.6890, -0.3258,  0.7129, -0.1919,  0.7297,  0.3793, -0.0537,  0.6240,
          0.7433, -0.5698,  0.6494, -0.3785, -0.6818,  0.5532,  0.9129, -0.4172,
          0.6378],
        [-1.6890, -0.0957, -0.0286, -0.6268, -0.0757, -0.3144,  0.1439,  0.1730,
          0.0885, -0.7273, -0.2664, -0.3785, -0.6818, -0.7164,  0.7455,  1.0684,
         -0.1951],
        [-1.7029,  0.1345, -1.0172, -0.8442, -0.6615, -0.7770,  0.7366, -0.7765,
         -0.4207, -0.8847, -0.5717, -0.3785, -0.6818, -0.6010, -0.5937, -0.4172,
         -0.8727],
        [-1.6821,  0.3646, -0.4170, -0.7355, -0.5883, -0.6845,  0.0451, -0.5391,
         -0.3116, -0.8847, -0.2664, -0.3785, -0.1046, -0.7164, -0.2589, -0.4172,
         -0.6609],
        [-1.6890,  0.5947,  0.5010, -0.6268,  0.2904, -0.0832,  2.0209, -0.0169,
          0.7433, -0.7273, -0.2664, -0.3785, -0.1046, -0.0239,  0.7455, -0.4172,
         -0.2515]], dtype=torch.float64)
tensor([[ 7.0000e+00,  2.5000e+01,  6.8000e+01,  6.0000e+00,  2.3000e+01,

In [6]:
# Get predictions
predictions = []
targets = []


for step, (x,y) in enumerate(player_data_loader):
    predictions.append(torch.mean(x, dim=1)) 
    targets.append(y)

predictions = torch.stack(predictions)
targets = torch.stack(targets)

print(predictions)
print(targets)

tensor([[[-1.7250,  1.7454,  0.4940,  ...,  0.5781,  0.1770,  0.6604]],

        [[-1.6904,  0.1345, -0.0498,  ...,  0.3103, -0.1201, -0.2685]],

        [[-1.6932,  0.3646, -0.1910,  ...,  0.0089, -0.1201, -0.4520]],

        ...,

        [[ 1.3739, -1.0162,  0.2821,  ..., -0.4932,  1.0684, -0.3842]],

        [[ 1.3808, -0.7861,  0.4940,  ...,  0.7790, -0.1201,  0.4797]],

        [[ 1.4113, -1.2464,  0.3104,  ...,  2.9551,  0.4741,  0.3809]]],
       dtype=torch.float64)
tensor([[[-1.7098,  2.4359,  0.4304,  ..., -0.5937, -0.4172, -0.1951]],

        [[-1.7029,  0.8249,  0.0067,  ..., -0.5937, -0.4172, -0.2798]],

        [[-1.7167,  1.0550,  0.8894,  ...,  0.4107, -0.4172,  0.5813]],

        ...,

        [[ 0.9834, -0.3258,  0.2892,  ..., -0.5937, -0.4172, -0.2939]],

        [[ 0.9904, -0.0957,  0.7129,  ...,  0.7455, -0.4172,  0.6942]],

        [[ 1.0180, -0.5560,  0.7129,  ...,  3.9260, -0.4172,  1.1883]]],
       dtype=torch.float64)


In [7]:
# Define loss
def RMSELoss(yhat, y):
    return torch.sqrt(torch.mean(yhat-y,axis=0)**2)

In [8]:
# Calculate the loss over all samples
#print(predictions)

loss = RMSELoss(predictions, targets)

print(predictions.shape)
print(targets.shape)



print(loss)
print(loss.mean())

torch.Size([3590, 1, 17])
torch.Size([3590, 1, 17])
tensor([[0.0608, 0.6903, 0.1120, 0.1590, 0.1495, 0.1621, 0.0560, 0.1754, 0.1415,
         0.1465, 0.1401, 0.1014, 0.1146, 0.1381, 0.1317, 0.0879, 0.1539]],
       dtype=torch.float64)
tensor(0.1600, dtype=torch.float64)
